# Diseases Mining from Pubtator Central

### Preqrequisites Libraries

In [1]:
from bs4 import BeautifulSoup
import requests
import numpy as np
import pandas as pd
import calendar
import time

### Mining Config

In [2]:
url = "https://www.ncbi.nlm.nih.gov/research/pubtator-api/publications/export/biocxml?pmids={}"

pubs_df = pd.read_csv("./data/1676879038-pubs-breast-cancer-4860.csv")

### Functions

In [3]:
logs_str = "Logs: {}"

def get_disease_from_pubs(row):
    print(logs_str.format("Getting disease from pub: {}".format(row['PMID'])))

    disease = np.array([])

    metadata_url = url.format(row['PMID'])
    metadata_response = requests.get(metadata_url)

    metadata_soup = BeautifulSoup(metadata_response.content, 'lxml')
    disease_sections = metadata_soup.find_all('infon', {'key': 'type'})
    for disease_section in disease_sections:
        if(disease_section.get_text().strip().lower() == 'disease'):
            text = disease_section.parent.find('text').get_text().strip()
            new_disease = np.array([row['PMID'], text])
            disease = np.append(disease, new_disease)

    print(logs_str.format("Found: {} diseases".format(disease.size)))

    return disease

def get_disease(df, limit=0):
    print(logs_str.format("🔨 Getting disease from {} publications".format(df.shape[0])))
    disease = np.array([])
    for index, row in df.iterrows():
        if(limit > 0 and index >= limit):
            break

        print(logs_str.format("Iteration number: {}".format(index)))
        
        try:
            new_disease = get_disease_from_pubs(row)
            disease = np.append(disease, new_disease)
        except Exception as e:
            print(logs_str.format("Error: {}".format(e)))
            continue
    
    print("-" * 50)
    print(logs_str.format("🌟 Job done!"))
    
    return disease

def transorm_to_df(dis, columns):
    df = pd.DataFrame(dis.reshape(-1, 2), columns=columns)
    return df

### Mining

In [ ]:
diseases = get_disease(pubs_df)
diseases_df = transorm_to_df(diseases, ['PMID', 'Disease'])

diseases_df.dropna(inplace=True) # Remove NaN
diseases_df["Disease"] = diseases_df["Disease"].astype(str).str.lower() # Lowercase
diseases_df = diseases_df.drop_duplicates() # Remove duplicates

diseases_df.describe()

Logs: 🔨 Getting disease from 4860 publications
Logs: Iteration number: 0
Logs: Getting disease from pub: 36800640


/home/adrianfinantyo/miniconda3/lib/python3.10/site-packages/bs4/builder/__init__.py:545: XMLParsedAsHTMLWarning: It looks like you're parsing an XML document using an HTML parser. If this really is an HTML document (maybe it's XHTML?), you can ignore or filter this warning. If it's XML, you should know that using an XML parser will be more reliable. To parse this document as XML, make sure you have the lxml package installed, and pass the keyword argument `features="xml"` into the BeautifulSoup constructor.
  warnings.warn(


Logs: Found: 0 diseases
Logs: Iteration number: 1
Logs: Getting disease from pub: 36800620
Logs: Found: 0 diseases
Logs: Iteration number: 2
Logs: Getting disease from pub: 36800575
Logs: Found: 0 diseases
Logs: Iteration number: 3
Logs: Getting disease from pub: 36800561
Logs: Found: 0 diseases
Logs: Iteration number: 4
Logs: Getting disease from pub: 36800556
Logs: Found: 0 diseases
Logs: Iteration number: 5
Logs: Getting disease from pub: 36800552
Logs: Found: 0 diseases
Logs: Iteration number: 6
Logs: Getting disease from pub: 36800539
Logs: Found: 0 diseases
Logs: Iteration number: 7
Logs: Getting disease from pub: 36800443
Logs: Found: 0 diseases
Logs: Iteration number: 8
Logs: Getting disease from pub: 36800412
Logs: Found: 0 diseases
Logs: Iteration number: 9
Logs: Getting disease from pub: 36800370
Logs: Found: 0 diseases
Logs: Iteration number: 10
Logs: Getting disease from pub: 36800273
Logs: Found: 0 diseases
Logs: Iteration number: 11
Logs: Getting disease from pub: 368002

### Export Data to CSV

In [ ]:
file_path = "./data/"
ts = calendar.timegm(time.gmtime())
num_dis = diseases_df.shape[0]
file_name = "{}-diseases-{}.csv".format(ts, num_dis)

diseases_df.to_csv(file_path+file_name, index=False)